In [1]:
!pip install pymysql > /dev/null

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]


Saving mysql.json to mysql.json


In [24]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)
config['database'] = 'mcdb'
import pymysql
conn = pymysql.connect(
    host=config['host'], 
    user=config['user'], 
    password=config['password'], 
    database=config['database'], 
    port=config['port']
)

- 패스워드 암호화

In [25]:
import hashlib
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- Base64로 인코딩


In [26]:
import base64

base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [27]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [28]:
len(hashed_pwd)

44

In [29]:
import hashlib, base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [30]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

- users table에 패스워드 넣기 (uid + 21)


In [31]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',), ('djy',), ('eskim',), ('gdhong',), ('hjpark',), ('wjlee',))

In [32]:
sql_update = 'update users set pwd=%s where uid=%s;'
for line in results:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update, (pwd, uid))
conn.commit()

- email 추가

In [33]:
sql_email = 'update users set email=%s where uid=%s;'
for line in results:
    uid = line[0]
    email = uid + '@mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()

- 로그인

- CASE1. 아이디가 없는 경우

In [36]:
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'

In [37]:
# result가 Null
uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

- CASE2. 비밀번호가 틀린 경우

In [38]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [39]:
db_pwd = result[1]
hashed_pwd == gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Incorrect password!


- CASE3. OK


In [42]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Incorrect password!


- 3가지 경우를 합침

In [43]:
uid, pwd = 'djy1', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Bad uid!!!


- 예외처리

In [44]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
except:
    print('Bad uid!!!')

Login


In [45]:
cur.close()
conn.close()